Notebook outlining what functions from nltk will be useful for pre-processing the text and training the classification algorithm.

Expects a CSV file in the current directory containing a release's date, text, and classification (pos or neg) in each row.

In [17]:
import pandas as pd
import nltk
import re

In [7]:
release_df = pd.read_csv('textsdf.csv', index_col=0)


In [8]:
release_df[:10]

,date,release_text
0,"January 31, 2018",Information received since the Federal Open Ma...
1,"February 01, 2017",Information received since the Federal Open Ma...
2,"March 15, 2017",Information received since the Federal Open Ma...
3,"May 03, 2017",Information received since the Federal Open Ma...
4,"June 14, 2017",Information received since the Federal Open Ma...
5,"July 26, 2017",Information received since the Federal Open Ma...
6,"September 20, 2017",Information received since the Federal Open Ma...
7,"November 01, 2017",Information received since the Federal Open Ma...
8,"December 13, 2017",Information received since the Federal Open Ma...
9,"January 27, 2016",Information received since the Federal Open Ma...


In [11]:
release_df['date'][0]

'January 31, 2018'

In [15]:
release_df['year'] = release_df['date'].apply(lambda x: int(x[-4:]))

In [ ]:
release_df['month'] = release_df['date'].apply()

In [16]:
release_df[:5]

,date,release_text,year
0,"January 31, 2018",Information received since the Federal Open Ma...,2018
1,"February 01, 2017",Information received since the Federal Open Ma...,2017
2,"March 15, 2017",Information received since the Federal Open Ma...,2017
3,"May 03, 2017",Information received since the Federal Open Ma...,2017
4,"June 14, 2017",Information received since the Federal Open Ma...,2017


In [ ]:
#release_df['tokenized'] = release_df['text']